In [2]:
import pandas as pd      # the necessary library
import json              # json duh
from rich import print   # extra python package to giove std output colorful and more readable

## Json Into DataFrame

For now I want to discuss converting a json file into something a DataFrame can handle. Converting a CSV into a dataFrame is really straight forward, but Json is not really setup like a CSV. This isn't always convenient, but most data isn't typically stored in a way that makes it easy for us, but get used to it.  Much of your time in the research game will be converting data from on format to another. Whether you are removing anomalies, cleaning up bad values, or altering the format to use for a specific purpose, you will find yourself manipulating data to increase its usability for your purpose.

Having said that, we need to know how to convert data into a format that can imported correctly into a DataFrame. DataFrames are extremely popular across many python libraries, and since I like json, we need to have the ability to convert a json data file to a format readable by a dataFrame.

### Json

Json is a string file format represented as: `key:value` pairs. It is not columnar like dataFrames expect. To fix Json data so that a pandas dataFrame can read it we need to turn all of the `key:values` into parallel lists (when and if possible). This is done by creating "parallel arrays" or "parallel lists". What is a parallel array? A parallel array is an array that uses a single index value to access data for a single entity. Look at the image below: 

<img src="./images/parallel_arrays.png" width="400">

In the C++ world, we would define this as something similar to below:

```cpp
string Names[7];
int Age[7];
string Sex[7];

// load arrays
// then
// To print out data for each individual:
for(int i=0;i<7;i++){
    cout<<Names[i]<<" "<<Age[i]<<" "<<Sex[i]<<endl;
}
```

Index "i" access the data associated with the same individual, where a single "individuals" info is stored in three separate arrays:  `Name`, `Age`, and `Sex`, related together by index number. The array values could have been read in a variety of ways, but you must admit storing data in Json strings is pretty convenient. Look at the snippet below which is the Json version of the data above. 
```json
{
    "Name": [
        "Braund, Mr. Owen Harris",
        "Allen, Mr. William Henry",
        "Bonnell, Miss. Elizabeth",
    ],
    "Age": [22, 35, 58],
    "Sex": ["male", "male", "female"],
}
```

Please note, this is not the "typical" way in which data is stored in a json array. More likely the above data would be stored like below which is a list of objects, where each object has the data associated with each entity:

```json
[
    {
        "Name": "Braund, Mr. Owen Harris",
        "Age": 22,
        "Sex": "male"
    },
    {
        "Name": "Allen, Mr. William Henry",
        "Age": 35,
        "Sex": "male"
    },
    {
        "Name": "Bonnell, Miss. Elizabeth",
        "Age": 58,
        "Sex": "female"
    }
]
```

So using a typical json "list" of "objects" like below, lets convert it into something that can be loaded by a dataFrame. 

#### Typical Json to DataFrame Ready

When using a Python `dictionary of lists`, the dictionary keys will be used as `column headers` and the values in each list are the values placed the `columns` for the `DataFrame`. Given the following Json object, lets make it so a DataFrame can load it without issue.

```json
[
    {
        "city": "New York",
        "growth": 4.8,
        "latitude": 40.7127837,
        "longitude": -74.0059413,
        "population": 8405837,
        "rank": 1,
        "state": "New York"
    },
    {
        "city": "Los Angeles",
        "growth": 4.8,
        "latitude": 34.0522342,
        "longitude": -118.2436849,
        "population": 3884307,
        "rank": 2,
        "state": "California"
    },
    {
        "city": "Chicago",
        "growth": -6.1,
        "latitude": 41.8781136,
        "longitude": -87.6297982,
        "population": 2718782,
        "rank": 3,
        "state": "Illinois"
    },
    ...
]
```

This is found [here](./cities_latlon_w_pop.json) in its full form. 

Let's start by loading the file:

In [3]:
import os
print(os.getcwd())

with open("../../Resources/01_Data/cities_latlon_w_pop.json") as f:
    cities = json.load(f)

# print first 3 cities
print(cities[:3])

/Users/griffin/Dropbox/_Courses/4553-Spatial-DS/Lectures/03_Pandas

[
    {
        'city': 'New York',
        'growth': 4.8,
        'latitude': 40.7127837,
        'longitude': -74.0059413,
        'population': 8405837,
        'rank': 1,
        'state': 'New York'
    },
    {
        'city': 'Los Angeles',
        'growth': 4.8,
        'latitude': 34.0522342,
        'longitude': -118.2436849,
        'population': 3884307,
        'rank': 2,
        'state': 'California'
    },
    {
        'city': 'Chicago',
        'growth': -6.1,
        'latitude': 41.8781136,
        'longitude': -87.6297982,
        'population': 2718782,
        'rank': 3,
        'state': 'Illinois'
    }
]

#### Data Loaded Now Convert

We loaded the data, now let's create a new Json object with the seven parallel arrays needed, one for each of the data elements. The keys are `city`, `growth`, ... `state`. I will grab them programmatically, then  create the parallel array version of the json object.



In [5]:
# grab keys from 1st entry
keys = cities[0].keys()

# create a new dictionary (synonymous with json)
parallelCityData = {}

# iterate over keys and create a list for every key
for key in keys:
    parallelCityData[key] = []

# Note: the keys may not print in the same order you added them. 
# This is ok behavior since dictionaries are not ordered.
print(parallelCityData)

{
    'city': [],
    'growth': [],
    'latitude': [],
    'longitude': [],
    'population': [],
    'rank': [],
    'state': []
}

Now we have the new structure (parallel lists of data) , lets process the file data to load it up:

In [9]:
# Every loop a new city is loaded into the "city" variable
for city in cities:

    # Loop over the Keys that exist in our new parallel structure
    for key in parallelCityData:

        # if the key from "parallCityData" is in the "city" object
        # then add that data element to the "parallelCityData" object
        if key in city:
            parallelCityData[key].append(city[key])
        else:
            parallelCityData[key].append(None)

# print first five data elements in the newly formatted object
for key in parallelCityData:
    print(parallelCityData[key][:5])



['New York', 'Los Angeles', 'Chicago', 'Houston', 'Philadelphia']

[4.8, 4.8, -6.1, 11, 2.6]

[40.7127837, 34.0522342, 41.8781136, 29.7604267, 39.9525839]

[-74.0059413, -118.2436849, -87.6297982, -95.3698028, -75.1652215]

[8405837, 3884307, 2718782, 2195914, 1553165]

[1, 2, 3, 4, 5]

['New York', 'California', 'Illinois', 'Texas', 'Pennsylvania']

Now we can load it into a dataFrame with no problem! 

In [10]:
df = pd.DataFrame(parallelCityData)
print(df)

df.info()

city  growth   latitude   longitude  population  rank  \
0         New York     4.8  40.712784  -74.005941     8405837     1   
1      Los Angeles     4.8  34.052234 -118.243685     3884307     2   
2          Chicago    -6.1  41.878114  -87.629798     2718782     3   
3          Houston    11.0  29.760427  -95.369803     2195914     4   
4     Philadelphia     2.6  39.952584  -75.165222     1553165     5   
...            ...     ...        ...         ...         ...   ...   
2995  Spanish Fork    78.1  40.114955 -111.654923       36956   998   
2996        Beloit     2.9  42.508348  -89.031776       36888   999   
2997   Panama City     0.1  30.158813  -85.660206       36877  1000   
2998     Anchorage    15.4  61.218056 -149.900278      300950    63   
2999      Honolulu    -6.2  21.306944 -157.858333      347884    54   

             state  
0         New York  
1       California  
2         Illinois  
3            Texas  
4     Pennsylvania  
...            ...  
2995          Utah  
2996     Wisconsin  
2997       Florida  
2998        Alaska  
2999        Hawaii  

[3000 rows x 7 columns]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   city        3000 non-null   object 
 1   growth      3000 non-null   float64
 2   latitude    3000 non-null   float64
 3   longitude   3000 non-null   float64
 4   population  3000 non-null   int64  
 5   rank        3000 non-null   int64  
 6   state       3000 non-null   object 
dtypes: float64(3), int64(2), object(2)
memory usage: 164.2+ KB
